### Import stuff

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
from scipy.signal import savgol_filter
from scipy.spatial import distance
from librosa.sequence import dtw
from tqdm import trange, tqdm
import networkx as nx
from shapely.geometry import Polygon, MultiPolygon, Point, MultiLineString, LineString, shape, JOIN_STYLE
from shapely.geometry.polygon import LinearRing
from shapely.ops import snap, unary_union
import geopandas as gpd
from glob import glob

import sys
sys.path.append('../meandergraph/')
import meandergraph as mg

In [ ]:
# Enable automatic module reloading
%load_ext autoreload
%autoreload 2

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Set backend to display mpl plots in separate interactive window
%matplotlib qt

### Load data

In [ ]:
lbs = sorted(glob("../data/lb*.shp"))
rbs = sorted(glob("../data/rb*.shp"))

In [ ]:
X1 = []
X2 = []
Y1 = []
Y2 = []
for fname in lbs:
    lb = gpd.read_file(fname)
    X2.append(lb['geometry'][0].xy[0])
    Y2.append(lb['geometry'][0].xy[1])
for fname in rbs:
    rb = gpd.read_file(fname)
    X1.append(rb['geometry'][0].xy[0])
    Y1.append(rb['geometry'][0].xy[1])

In [ ]:
# plot banklines
plt.figure()
for i in range(len(X1)):
    plt.plot(X1[i], Y1[i], 'k', linewidth = 0.5)
    plt.plot(X2[i], Y2[i], 'b', linewidth = 0.5)
plt.axis('equal');

### Define segment of interest

In [ ]:
points = plt.ginput(n=2) 
# click twice to select start- and endpoints on first bankline (does not matter which one)

In [ ]:
# comment this cell out if you selected points in the previous cell using ginput
points = [(286850.39156848995, -1660631.8404876508),
 (284610.488612104, -1650178.960024516)]

In [ ]:
from scipy import signal, spatial
cl_points = np.vstack((X1[0], Y1[0])).T # coordinates of first centerlines
tree = spatial.KDTree(cl_points)

plt.figure()
for i in range(len(X1)):
    plt.plot(X1[i], Y1[i], 'k', linewidth = 0.5)
    plt.plot(X2[i], Y2[i], 'b', linewidth = 0.5)
plt.axis('equal')
plt.plot(X1[0][tree.query(points[0])[1]], Y1[0][tree.query(points[0])[1]], 
         'ro', zorder=10000)
plt.plot(X1[0][tree.query(points[1])[1]], Y1[0][tree.query(points[1])[1]], 
         'ro', zorder=10000)

### Correlate long banklines and create coordinate arrays for segment of interest

In [ ]:
first_index = tree.query(points[0])[1]
last_index = tree.query(points[1])[1]

first_channel = 0
last_channel = 31

# correlate all lines:    
P1 = []
Q1 = []
for i in trange(len(X1) - 1):
    p, q = mg.correlate_curves(X1[i], X1[i+1], Y1[i], Y1[i+1])
    P1.append(p)
    Q1.append(q)

# correlate all lines:    
P2 = []
Q2 = []
for i in trange(len(X2) - 1):
    p, q = mg.correlate_curves(X2[i], X2[i+1], Y2[i], Y2[i+1])
    P2.append(p)
    Q2.append(q)
    
indices1, x, y = mg.find_indices(first_index, X1, Y1, P1, Q1)
indices2, x, y = mg.find_indices(last_index, X1, Y1, P1, Q1)
for i in range(len(X1)):
    X1[i] = X1[i][indices1[i] : indices2[i]+1]
    Y1[i] = Y1[i][indices1[i] : indices2[i]+1]

indices1, x, y = mg.find_indices(first_index, X2, Y2, P2, Q2)
indices2, x, y = mg.find_indices(last_index, X2, Y2, P2, Q2)
for i in range(len(X2)):
    X2[i] = X2[i][indices1[i] : indices2[i]+1]
    Y2[i] = Y2[i][indices1[i] : indices2[i]+1]

### Resample and correlate centerlines and banklines

In [ ]:
# resample centerlines to ds = 2.0 meters:
for i in range(len(X1)):
    x,y,dx,dy,ds,s = mg.resample_centerline(X1[i], Y1[i], 2.0)
    X1[i] = x
    Y1[i] = y
for i in range(len(X2)):
    x,y,dx,dy,ds,s = mg.resample_centerline(X2[i], Y2[i], 2.0)
    X2[i] = x
    Y2[i] = y

P1, Q1 = mg.correlate_set_of_curves(X1, Y1)
P2, Q2 = mg.correlate_set_of_curves(X2, Y2)

In [ ]:
plt.figure()
plt.plot(X1[10], Y1[10])
plt.plot(X2[10], Y2[10])
plt.axis('equal');

### Create centerline- and bank graphs

In [ ]:
ts = len(X1)
timesteps = [1.0] * len(X1) # create a list of timesteps; elements have units of years. In this case the timestep is 1 year
graph1 = mg.create_graph_from_channel_lines(X1[:ts], Y1[:ts], P1[:ts-1], Q1[:ts-1], n_points=20, max_dist=1000, smoothing_factor = 51, remove_cutoff_edges=True, timesteps = timesteps)
graph2 = mg.create_graph_from_channel_lines(X2[:ts], Y2[:ts], P2[:ts-1], Q2[:ts-1], n_points=20, max_dist=1000, smoothing_factor= 51, remove_cutoff_edges=True, timesteps = timesteps)

In [ ]:
graph1 = mg.remove_high_density_nodes(graph1, min_dist = 20, max_dist = 60)
graph2 = mg.remove_high_density_nodes(graph2, min_dist = 20, max_dist = 60)

### View node attributes

In [ ]:
graph1.nodes[0]

### Plot one of the graphs

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
mg.plot_graph(graph2, ax)
plt.axis('equal');

In [ ]:
# save graphs
nx.write_gpickle(graph1, "graph1.gpickle")
nx.write_gpickle(graph2, "graph2.gpickle")

In [ ]:
cutoff_area = 1*1e6

fig = plt.figure()
ax = fig.add_subplot(111)
bars, chs, all_chs, jumps, cutoffs = mg.plot_bars_from_banks(graph1, graph2, cutoff_area, ax)

### Create and plot scrolls and bars (connected scrolls)

In [ ]:
cutoff_area = 1*1e6
scrolls, scroll_ages, cutoffs, all_bars_graph = mg.create_scrolls_and_find_connected_scrolls(graph1, graph2, cutoff_area)
plt.axis('equal');

### Create 'bar graphs'

In [ ]:
# create bars and bar graphs
min_area = 1000
wbars, poly_graph_1, poly_graph_2 = mg.create_polygon_graphs_and_bar_graphs(graph1, graph2, all_bars_graph, 
                                                    scrolls, scroll_ages, X1, Y1, X2, Y2, min_area)

### Plot 'bar' graphs, colored by an attribute

#### Color by migration rate

In [ ]:
fig = plt.figure(figsize = (12, 12)) 
ax = fig.add_subplot(111)
mg.plot_bar_graphs(graph1, graph2, wbars, cutoffs, X1, Y1, X2, Y2, 300, -500, 500, 'migration', ax)
ax.set_adjustable("box")
ax.axis('equal')
fig.tight_layout()

#### Color by age

In [ ]:
# plot them, using migration rate
fig = plt.figure(figsize = (12, 12)) 
ax = fig.add_subplot(111)
mg.plot_bar_graphs(graph1, graph2, wbars, cutoffs, X1, Y1, X2, Y2, 300, 0, len(X1), 'age', ax)
ax.set_adjustable("box")
ax.axis('equal')
fig.tight_layout()

### View polygon graph node attributes

In [ ]:
poly_graph_1.nodes[1000]

### Plot radial lines for line graphs

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
cmap = plt.get_cmap("tab10")
for node in tqdm(graph1.graph['start_nodes']):
    path, path_ages = mg.find_radial_path(graph1, node)
    for i in range(len(path)-1):
        if graph1[path[i]][path[i+1]]['direction'] == 1:   # erosion on the right bank
            ax.plot(graph1.graph['x'][[path[i], path[i+1]]], 
                    graph1.graph['y'][[path[i], path[i+1]]], color=cmap(1))
        if graph1[path[i]][path[i+1]]['direction'] == -1:  # deposition on the right bank
            ax.plot(graph1.graph['x'][[path[i], path[i+1]]], 
                    graph1.graph['y'][[path[i], path[i+1]]], color=cmap(0))
plt.axis('equal');

### Plot migration rate map for one bar

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.fill(wbars[2].polygon.exterior.xy[0], wbars[2].polygon.exterior.xy[1])
mg.plot_migration_rate_map(wbars[2], graph1, graph2, vmin=-300, vmax=300, ax=ax)
plt.axis('equal');

### Plot age map for one bar

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
mg.plot_age_map(wbars[1], vmin=0, vmax=len(X1), ax=ax)
plt.axis('equal');

### Plot bar polygons and their numbers

In [ ]:
fig = plt.figure(figsize = (15, 9))
ax = fig.add_subplot(111)
for wbar in wbars:
   if wbar.polygon.geom_type == 'Polygon':
    wbar.add_bank_type() # add bank type
    if wbar.bank_type == 'right':
        plt.fill(wbar.polygon.exterior.xy[0], wbar.polygon.exterior.xy[1], color='r')
    if wbar.bank_type == 'left':
        plt.fill(wbar.polygon.exterior.xy[0], wbar.polygon.exterior.xy[1], color='b')
    count = 0
    for wbar in wbars:
        ax.text(wbar.polygon.centroid.x, wbar.polygon.centroid.y, str(count), fontsize = 16)
        count += 1
plt.axis('equal');

### Write shapefile

In [ ]:
polys = []
migr_rates = []
directions = []
ages = []
for node in poly_graph_1.nodes:
    polys.append(poly_graph_1.nodes[node]['poly'])
    migr_rates.append(poly_graph_1.nodes[node]['migr_rate']*poly_graph_1.nodes[node]['direction'])
    directions.append(poly_graph_1.nodes[node]['direction'])
    ages.append(poly_graph_1.nodes[node]['age'])
gdf = gpd.GeoDataFrame(polys, columns = ['geometry'])
gdf['migr_rate'] = migr_rates
gdf['direction'] = directions
gdf['age'] = ages
gdf['area'] = gdf.area
gdf.head()

In [ ]:
gdf.crs = {'init' :'epsg:32620'}

In [ ]:
gdf.to_file("mamore_right_bank.shp") # write shapefile